In [1]:
import torch
import pickle
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import os
import config
import utils
import main_transfer as mt


## SOURCE_DATASET 설정

In [2]:
# seed 고정
random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
# root directory 설정
SOURCE_DATASET = 'Computers'

data_root_dir = f'./data/{SOURCE_DATASET}/'

In [4]:
# 데이터 분할 
train_x, train_y, test_x, test_y = utils.load_data(data_root_dir, model_name='LSTM_FCNs')  # shape=(num_of_instance, input_dims, time_steps)
        
split_ratio = 0.2
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=split_ratio, shuffle=True)

input_size = train_x.shape[1]
num_classes = len(np.unique(train_y))

# normalization
new_dir_path = f'./scaler/{SOURCE_DATASET}/'
os.makedirs(new_dir_path, exist_ok=True)

scaler_x_path = f'./scaler/{SOURCE_DATASET}/minmax_scaler_x.pkl'
train_x, valid_x = utils.get_train_val_data(train_x, valid_x, scaler_x_path)


(250, 1, 720)
(250,)
(250, 1, 720)
(250,)
inputSize(train_x.shape[1]): 1
sequenceLenth (train_x.shape[2]): 720
Save MinMaxScaler in path: ./scaler/Computers/minmax_scaler_x.pkl


In [5]:
# build model
model_name = 'LSTM_FCNs'
model_params = config.model_config[model_name]

model_params['parameter']['input_size'] = input_size
model_params['parameter']['num_classes'] = num_classes
model_params['best_model_path'] = f'./ckpt/{SOURCE_DATASET}/lstm_fcn.pt'

data_source = mt.Transferlearning(model_params,'self') ## 자기자신 데이터로 학습 하는 option : self
data_source.build_model()

best_model = data_source.train_model(train_x, train_y, valid_x, valid_y)  # 모델 학습

Set start class as zero
Set start class as zero

Epoch 1/300
train Loss: 0.6915 Acc: 0.5000
val Loss: 0.6908 Acc: 0.5400

Epoch 10/300
train Loss: 0.6273 Acc: 0.6350
val Loss: 0.7139 Acc: 0.4600

Epoch 20/300
train Loss: 0.6125 Acc: 0.6400
val Loss: 0.6184 Acc: 0.7200

Epoch 30/300
train Loss: 0.6062 Acc: 0.6400
val Loss: 0.5675 Acc: 0.7400

Epoch 40/300
train Loss: 0.5865 Acc: 0.6350
val Loss: 0.5581 Acc: 0.7600

Epoch 50/300
train Loss: 0.5762 Acc: 0.6650
val Loss: 0.5411 Acc: 0.7200

Epoch 60/300
train Loss: 0.5632 Acc: 0.6750
val Loss: 0.5288 Acc: 0.7200

Epoch 70/300
train Loss: 0.5616 Acc: 0.7100
val Loss: 0.5241 Acc: 0.7600

Epoch 80/300
train Loss: 0.5502 Acc: 0.7300
val Loss: 0.5141 Acc: 0.7600

Epoch 90/300
train Loss: 0.5476 Acc: 0.7200
val Loss: 0.4996 Acc: 0.7600

Epoch 100/300
train Loss: 0.5474 Acc: 0.7350
val Loss: 0.5033 Acc: 0.7600

Epoch 110/300
train Loss: 0.5404 Acc: 0.7450
val Loss: 0.5082 Acc: 0.6800

Epoch 120/300
train Loss: 0.5310 Acc: 0.7800
val Loss: 0.4983 

In [6]:
os.makedirs(f'./ckpt/{SOURCE_DATASET}/', exist_ok=True)
data_source.save_model(best_model, best_model_path=model_params["best_model_path"])  # 모델 저장
print(num_classes)


2


## Transfer to Target Dataset

In [7]:
# =============================================================================
# Test / Trasnfer
# =============================================================================
import config
import utils
import torch.nn as nn
TARGET_DATASET = 'ScreenType'
MODE = 'TEST'

In [8]:
# load raw data
data_root_dir = f'./data/{TARGET_DATASET}/'
train_x, train_y, test_x, test_y = utils.load_data(data_root_dir, model_name='LSTM_FCNs')  # shape=(num_of_instance, input_dims, time_steps)

input_size = train_x.shape[1]
num_classes = len(np.unique(train_y))

# normalization

split_ratio = 0.2
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=split_ratio, shuffle=True)

(375, 1, 720)
(375,)
(375, 1, 720)
(375,)
inputSize(train_x.shape[1]): 1
sequenceLenth (train_x.shape[2]): 720


In [9]:
scaler_x_path = f'./scaler/{TARGET_DATASET}/minmax_scaler_x.pkl'
train_x, valid_x = utils.get_train_val_data(train_x, valid_x, scaler_x_path)

model_name = 'LSTM_FCNs'
model_params = config.model_config[model_name]
num_classes = len(np.unique(train_y))

model_params['parameter']['input_size'] = input_size
model_params['parameter']['num_classes'] = num_classes
#model_params['parameter']['num_classes'] = 3
model_params['best_model_path'] = f'./ckpt/{SOURCE_DATASET}/lstm_fcn.pt' ##SOURCE DATSET 

Save MinMaxScaler in path: ./scaler/ScreenType/minmax_scaler_x.pkl


In [10]:
data_target = mt.Transferlearning(model_params,'transfer') ## Sourece 로 부터 데이터 받아서 Target 에 적용
data_target_self = mt.Transferlearning(model_params,'self') ## 자기자신 처음부터 다시 학습

In [11]:
best_model = data_target.train_model(train_x, train_y, valid_x, valid_y,'target')

Set start class as zero
Set start class as zero
model fc layer output change

Epoch 1/300
train Loss: 1.0960 Acc: 0.4267
val Loss: 1.1043 Acc: 0.3600

Epoch 10/300
train Loss: 1.0513 Acc: 0.4600
val Loss: 1.0724 Acc: 0.3867

Epoch 20/300
train Loss: 1.0362 Acc: 0.4733
val Loss: 1.0677 Acc: 0.3733

Epoch 30/300
train Loss: 1.0276 Acc: 0.4600
val Loss: 1.0666 Acc: 0.3733

Epoch 40/300
train Loss: 1.0154 Acc: 0.4633
val Loss: 1.0654 Acc: 0.3867

Epoch 50/300
train Loss: 1.0059 Acc: 0.4633
val Loss: 1.0623 Acc: 0.3733

Epoch 60/300
train Loss: 0.9974 Acc: 0.4667
val Loss: 1.0607 Acc: 0.3600

Epoch 70/300
train Loss: 0.9867 Acc: 0.4833
val Loss: 1.0558 Acc: 0.3733

Epoch 80/300
train Loss: 0.9739 Acc: 0.4967
val Loss: 1.0516 Acc: 0.4000

Epoch 90/300
train Loss: 0.9592 Acc: 0.5067
val Loss: 1.0480 Acc: 0.4133

Epoch 100/300
train Loss: 0.9460 Acc: 0.5100
val Loss: 1.0372 Acc: 0.4133

Epoch 110/300
train Loss: 0.9344 Acc: 0.5200
val Loss: 1.0260 Acc: 0.4400

Epoch 120/300
train Loss: 0.9236 

In [12]:
os.makedirs(f'./ckpt/{TARGET_DATASET}/', exist_ok=True)
data_target.save_model(best_model, best_model_path=f'./ckpt/{TARGET_DATASET}/lstm_fcn_transfer.pt')  # 모델 저장
print(num_classes)


3


In [13]:
best_model_self = data_target_self.train_model(train_x, train_y, valid_x, valid_y,'source')

Set start class as zero
Set start class as zero

Epoch 1/300
train Loss: 1.1007 Acc: 0.3200
val Loss: 1.1078 Acc: 0.3333

Epoch 10/300
train Loss: 1.0567 Acc: 0.4400
val Loss: 1.0871 Acc: 0.4267

Epoch 20/300
train Loss: 1.0353 Acc: 0.4500
val Loss: 1.0793 Acc: 0.3600

Epoch 30/300
train Loss: 1.0220 Acc: 0.4533
val Loss: 1.0776 Acc: 0.3600

Epoch 40/300
train Loss: 1.0118 Acc: 0.4700
val Loss: 1.0730 Acc: 0.3600

Epoch 50/300
train Loss: 1.0058 Acc: 0.4667
val Loss: 1.0677 Acc: 0.3733

Epoch 60/300
train Loss: 0.9968 Acc: 0.5133
val Loss: 1.0536 Acc: 0.4267

Epoch 70/300
train Loss: 0.9815 Acc: 0.5167
val Loss: 1.0528 Acc: 0.4133

Epoch 80/300
train Loss: 0.9746 Acc: 0.5233
val Loss: 1.0471 Acc: 0.4667

Epoch 90/300
train Loss: 0.9630 Acc: 0.5533
val Loss: 1.0354 Acc: 0.4667

Epoch 100/300
train Loss: 0.9544 Acc: 0.5433
val Loss: 1.0317 Acc: 0.4800

Epoch 110/300
train Loss: 0.9509 Acc: 0.5300
val Loss: 1.0300 Acc: 0.4667

Epoch 120/300
train Loss: 0.9378 Acc: 0.5467
val Loss: 1.0210 

In [14]:
os.makedirs(f'./ckpt/{TARGET_DATASET}/', exist_ok=True)
data_target_self.save_model(best_model_self, best_model_path=f'./ckpt/{TARGET_DATASET}/lstm_fcn_self.pt')  # 모델 저장
print(num_classes)


3
